# 🧠 Model 3 Section 1: Homework
> ## Building a Full RAG Pipeline

> **🎯 Today’s Goal**: Combine all the parts from Section 1. We will use the **Retriever** (MiniLM) and the **Generator** (DistilBERT) to build a complete, end-to-end Retrieval-Augmented Generation (RAG) system.

---

###  recap: The Two Parts of Our RAG System

1.  **The Retriever (Part 2)** 🔎
    * **Model:** `all-MiniLM-L6-v2`
    * **Job:** To turn a text query into a vector and use **semantic search** to find the *most relevant* piece of context from our knowledge base.

2.  **The Generator (Part 3)** ✍️
    * **Model:** `distilbert-base-cased-distilled-squad`
    * **Job:** To take a `question` and a `context` and **extract** the specific answer from within the context.

Today, we connect them. The output of the Retriever becomes the input for the Generator.

---


---

### 🧠 Step 1: Load Models and Knowledge

Now, let's load both of our specialized models and define our knowledge base. We have one model for retrieving and one for generating.

---

In [37]:
# Code Cell 2: Load Models (The "CPU/GPU Split" Fix)
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import torch

# --- 1. Load the Retriever (MiniLM) on the CPU ---
# We explicitly tell it to use the 'cpu'.
# This is fast enough for a retriever and saves all our VRAM.
retriever_model = SentenceTransformer(
    'all-MiniLM-L6-v2',
    device='cpu'  # Force to CPU
)
print(f"✅ Retriever model (MiniLM) loaded. Using device: cpu")


# --- 2. Load the Generator (DistilBERT) on the GPU ---
# We check if a GPU is available and set the device index
# 0 = first GPU, -1 = CPU
pipeline_device = 0 if torch.cuda.is_available() else -1

generator_model = pipeline("question-answering",
                           model="distilbert-base-cased-distilled-squad",
                           device=pipeline_device) # Use GPU if available

print(f"✅ Generator model (DistilBERT) loaded.")
if pipeline_device == 0:
    print("   -> Running on GPU (Good!)")
else:
    print("   -> WARNING: Running on CPU (Will be slow, but should work)")

✅ Retriever model (MiniLM) loaded. Using device: cpu


Device set to use cuda:0


✅ Generator model (DistilBERT) loaded.
   -> Running on GPU (Good!)


---

### 📚 Step 2: Define and Encode Knowledge Base

Here is our simple knowledge base. We will give this "long-term memory" to our AI agent.

---

In [38]:
# Code Cell 3: Define Knowledge Base
# This is the "long-term memory" of our agent
knowledge_base = [
    # Original sentences
    "Buddy is a 3-year-old Golden Retriever who loves to play fetch.",
    "The capital of France is Paris, which is known for the Eiffel Tower.",
    "Python is an interpreted, high-level, general-purpose programming language.",
    "The first person to walk on the Moon was Neil Armstrong in 1969.",
    "Climate change is the long-term alteration of temperature and typical weather patterns.",

]


print(f"📚 Knowledge base created with {len(knowledge_base)} documents.")

📚 Knowledge base created with 5 documents.


---

### ✏️ Task 1: Encode Your Knowledge

Your first task is to use the **Retriever model** (`retriever_model`) to encode all the documents in your `knowledge_base`.

**Your Goal:** Create a variable called `knowledge_embeddings` that holds the vector representations of all your documents.

---

In [39]:
# Code Cell 4: Task 1 - Encode Knowledge
print("--- Task 1: Encoding Knowledge Base ---")


knowledge_embeddings = retriever_model.encode(knowledge_base,convert_to_tensor=True)

# --- Verification ---
if 'knowledge_embeddings' in locals() and knowledge_embeddings.shape[0] == len(knowledge_base):
    print("✅ Success! Knowledge base has been encoded.")
    print(f"   -> Embedding shape: {knowledge_embeddings.shape}")
else:
    print("⚠️ Task 1 not complete. 'knowledge_embeddings' not found or has wrong shape.")

--- Task 1: Encoding Knowledge Base ---
✅ Success! Knowledge base has been encoded.
   -> Embedding shape: torch.Size([5, 384])


---

### ✏️ Task 2: Build the Retriever Function

Now, let's build a function that performs the "R" (Retrieval) step. This function will take a user's `query` and find the most relevant document from our `knowledge_base`.

**Your Goal:** Complete the `retrieve_context` function.
1.  Encode the incoming `query` using the `retriever_model`.
2.  Use `util.pytorch_cos_sim` to compare the `query_embedding` to all `knowledge_embeddings`.
3.  Find the index of the highest-scoring document (use `torch.argmax`).
4.  Return the *text* of that document from the `knowledge_base`.

---

In [40]:
# Code Cell 5: Task 2 - Build the Retriever
print("--- Task 2: Building the Retriever ---")

def retrieve_context(query):
    # 1. Encode the query


    query_embedding = retriever_model.encode(query,convert_to_tensor=True)
    # 2. Compute cosine similarity

    cos_scores = util.pytorch_cos_sim(query_embedding, knowledge_embeddings)[0]



    # 3. Find the best match

    top_result_index = torch.argmax(cos_scores)

    # 4. Return the matching document text

    return knowledge_base[top_result_index]

# --- Verification ---
print("Testing retrieve_context('What is Python?')...")
retrieved = retrieve_context("What is Python?")
print(f"   -> Retrieved: '{retrieved}'")
if "Python" in retrieved:
    print("✅ Success! Retriever function works.")
else:
    print("⚠️ Retriever function failed to find the right document.")

--- Task 2: Building the Retriever ---
Testing retrieve_context('What is Python?')...
   -> Retrieved: 'Python is an interpreted, high-level, general-purpose programming language.'
✅ Success! Retriever function works.


---

### ✏️ Task 3: Build the Generator Function

Great! We have a function to get context. Now let's build a function for the "G" (Generation) step. This function will take a `question` and the `context` we just retrieved.

**Your Goal:** Complete the `generate_answer` function.
1.  Call the `generator_model` (which is a `pipeline` object).
2.  Pass the `question` and `context` to it.
3.  Return *only the answer* from the resulting dictionary (e.g., `result['answer']`).

---

In [41]:
# Code Cell 6: Task 3 - Build the Generator
print("\n--- Task 3: Building the Generator ---")

def generate_answer(question, context):
    # 1. Call the pipeline

    result = generator_model(question=question, context=context)

    # 2. Return the answer

    return result['answer']

# --- Verification ---
print("Testing generate_answer('What is Python?', '...')...")
test_context = "Python is a popular programming language."
test_question = "What is Python?"
answer = generate_answer(test_question, test_context)
print(f"   -> Question: '{test_question}'")
print(f"   -> Context: '{test_context}'")
print(f"   -> Answer: '{answer}'")

if "popular programming language" in answer:
    print("✅ Success! Generator function works.")
else:
    print("⚠️ Generator function failed to extract the answer.")


--- Task 3: Building the Generator ---
Testing generate_answer('What is Python?', '...')...
   -> Question: 'What is Python?'
   -> Context: 'Python is a popular programming language.'
   -> Answer: 'a popular programming language'
✅ Success! Generator function works.



---

### 🚀 Task 4: Build the Full RAG Pipeline!

This is the final step. Let's combine our two functions into a single, end-to-end RAG pipeline. This function will orchestrate the entire process.

**Your Goal:** Complete the `ask_rag_pipeline` function.
1.  Call your `retrieve_context` function to get the `best_context` for the `query`.
2.  Call your `generate_answer` function, passing in the *original* `query` and the `best_context` you just found.
3.  Return the final `answer`.

After you write the function, we'll test it with a query!

---

In [42]:
# Code Cell 7: Task 4 - Build the Full RAG Pipeline
print("\n--- Task 4: Building the Full RAG Pipeline ---")

def ask_rag_pipeline(query):
    # 1. Retrieve

    best_context = retrieve_context(query)

    # 2. Generate

    final_answer = generate_answer(question=query, context=best_context)

    # 3. Return
    return final_answer, best_context


# --- Verification ---
print("Testing the full RAG pipeline...")
print("Query: 'What is the capital of France?'")
final_answer, retrieved_context = ask_rag_pipeline("What is the capital of France?")

print(f"   -> Retrieved Context: '{retrieved_context}'")
print(f"   -> Final Answer: '{final_answer}'")

if final_answer.lower() == "paris":
    print("✅ Success! Your RAG pipeline is working!")
else:
    print("⚠️ RAG pipeline failed. Expected 'Paris'.")


--- Task 4: Building the Full RAG Pipeline ---
Testing the full RAG pipeline...
Query: 'What is the capital of France?'
   -> Retrieved Context: 'The capital of France is Paris, which is known for the Eiffel Tower.'
   -> Final Answer: 'Paris'
✅ Success! Your RAG pipeline is working!


---

### 🧪 Self-Assessment

Run this final cell to test your complete RAG pipeline. This assessment will:
1.  Add new information to the agent's knowledge base.
2.  Ask questions that *require* the RAG pipeline to work.
3.  It will check if your `retrieve_context` function finds the right document AND if your `generate_answer` function extracts the correct answer.

Good luck!

---

In [43]:
#@title # Code Cell 8: Self-Assessment
print("\n--- 🧪 Self-Assessment ---")

# We will add new documents to the knowledge base and test the agent.
# This simulates expanding the agent's memory.

try:
    # --- Setup for Test ---
    new_knowledge = [
    "The Great Wall of China stretches over 13,000 miles and was built to protect against invasions.",
    "Water boils at 100 degrees Celsius at standard atmospheric pressure.",
    "Albert Einstein developed the theory of relativity, one of the two pillars of modern physics.",
    "The Amazon Rainforest is the largest tropical rainforest in the world, spanning several South American countries.",
    "Mount Everest, located in the Himalayas, is the tallest mountain on Earth above sea level.",
    "Shakespeare wrote many famous plays including Hamlet, Macbeth, and Romeo and Juliet.",
    "The Pacific Ocean is the largest and deepest ocean on Earth.",
    "Electricity is the flow of electric charge, typically carried by electrons through a conductor.",
    "The human heart has four chambers: two atria and two ventricles.",
    "Light travels at approximately 299,792 kilometers per second in a vacuum.",
    "The Nile River is the longest river in Africa, flowing through multiple countries before reaching the Mediterranean Sea.",
    "Venus is the second planet from the Sun and has a very thick atmosphere composed mostly of carbon dioxide.",
    "Vaccines stimulate the immune system to provide protection against infectious diseases.",
    "The first programmable computer was developed by Charles Babbage in the 19th century.",
    "The Mona Lisa is a famous painting created by Leonardo da Vinci and is displayed in the Louvre Museum.",
    "Dinosaurs were a diverse group of reptiles that lived millions of years ago and are now extinct.",
    "Mount Fuji is the highest volcano in Japan and a popular tourist destination.",
    "The Sahara Desert is the largest hot desert in the world, located in northern Africa.",
    "Photosynthesis is the process by which plants convert sunlight into chemical energy.",
    "The Statue of Liberty, located in New York Harbor, was a gift from France to the United States."
    ]
    # Update all the pieces:
    # 1. Update the text list
    knowledge_base.extend(new_knowledge)
    # 2. Update the embeddings (re-encode everything)
    knowledge_embeddings = retriever_model.encode(knowledge_base, convert_to_tensor=True)

    print(f"📚 Agent memory updated. Total documents: {len(knowledge_base)}")

    # --- Test Cases ---
    extra_test_cases = [
    {
        "query": "Who developed the theory of relativity?",
        "expected_context_keyword": "Einstein",
        "expected_answer_keyword": "Albert Einstein"
    },
    {
        "query": "Which river is the longest in Africa?",
        "expected_context_keyword": "Nile River",
        "expected_answer_keyword": "Nile River"
    },
    {
        "query": "What is the tallest mountain on Earth?",
        "expected_context_keyword": "Mount Everest",
        "expected_answer_keyword": "Mount Everest"
    },
    {
        "query": "Which ocean is the largest on Earth?",
        "expected_context_keyword": "Pacific Ocean",
        "expected_answer_keyword": "Pacific Ocean"
    },
    {
        "query": "What is the process by which plants convert sunlight into energy?",
        "expected_context_keyword": "Photosynthesis",
        "expected_answer_keyword": "Photosynthesis"
    },
    {
        "query": "Who painted the Mona Lisa?",
        "expected_context_keyword": "Mona Lisa",
        "expected_answer_keyword": "Leonardo da Vinci"
    },
    {
        "query": "Which desert is the largest hot desert in the world?",
        "expected_context_keyword": "Sahara Desert",
        "expected_answer_keyword": "Sahara Desert"
    },
    {
        "query": "What is the speed of light in a vacuum?",
        "expected_context_keyword": "Light",
        "expected_answer_keyword": "299,792 kilometers per second"
    },
    {
        "query": "Which planet is second from the Sun and has a thick atmosphere?",
        "expected_context_keyword": "Venus",
        "expected_answer_keyword": "Venus"
    }
]


    score = 0
    total = len(extra_test_cases) * 2
    for i, test in enumerate(extra_test_cases):

        print(f"\n--- Test Case {i+1} ---")
        query = test["query"]
        print(f"Query: \"{query}\"")

        # Run the full pipeline
        answer, context = ask_rag_pipeline(query)

        # Check Retrieval
        print(f"   -> 🔎 Retrieved: '{context}'")
        if test["expected_context_keyword"] in context:
            print("   -> ✅ Retrieval Correct!")
            score += 1
        else:
            print(f"   -> ❌ Retrieval Failed. Expected context with: '{test['expected_context_keyword']}'")

        # Check Generation
        print(f"   -> ✍️ Answer: '{answer}'")
        if test["expected_answer_keyword"].lower() in answer.lower():
            print("   -> ✅ Generation Correct!")
            score += 1
        else:
            print(f"   -> ❌ Generation Failed. Expected answer with: '{test['expected_answer_keyword']}'")

    # Final Score
    print(f"\n--- 🏁 Assessment Complete ---")
    print(f"🎯 Your Final Score: {score} / {total}")
    if score == total:
        print("🎉🎉🎉 Perfect! You have successfully built and tested a full RAG pipeline!")
    elif score >= total // 2:
        print("👍 Great job! Your pipeline is working. Review any failed tests to see what happened.")
    else:
        print("🔧 Keep trying! Check your functions in Tasks 2, 3, and 4.")

except Exception as e:
    print(f"\n--- ⚠️ Assessment Failed ---")
    print(f"An error occurred: {e}")
    print("Please check your code in all tasks and try again.")


--- 🧪 Self-Assessment ---
📚 Agent memory updated. Total documents: 25

--- Test Case 1 ---
Query: "Who developed the theory of relativity?"
   -> 🔎 Retrieved: 'Albert Einstein developed the theory of relativity, one of the two pillars of modern physics.'
   -> ✅ Retrieval Correct!
   -> ✍️ Answer: 'Albert Einstein'
   -> ✅ Generation Correct!

--- Test Case 2 ---
Query: "Which river is the longest in Africa?"
   -> 🔎 Retrieved: 'The Nile River is the longest river in Africa, flowing through multiple countries before reaching the Mediterranean Sea.'
   -> ✅ Retrieval Correct!
   -> ✍️ Answer: 'The Nile River'
   -> ✅ Generation Correct!

--- Test Case 3 ---
Query: "What is the tallest mountain on Earth?"
   -> 🔎 Retrieved: 'Mount Everest, located in the Himalayas, is the tallest mountain on Earth above sea level.'
   -> ✅ Retrieval Correct!
   -> ✍️ Answer: 'Mount Everest'
   -> ✅ Generation Correct!

--- Test Case 4 ---
Query: "Which ocean is the largest on Earth?"
   -> 🔎 Retrieved: 